In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/titanic/

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, roc_auc_score, recall_score, precision_score

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
train_orig = train.copy()
test_orig = test.copy()
submission_orig = submission.copy()

In [ ]:
train.shape, test.shape, train.index, test.index

In [ ]:
train.columns.difference(test.columns), train.columns.intersection(submission.columns)

In [ ]:
train.duplicated().sum(), test.duplicated().sum()
# there are no duplicate rows in both train n test

In [ ]:
train["source"] = "train"
test["source"] = "test"
data = pd.concat([train,test])
data_orig = data.copy()
data.head()

In [ ]:
data.info()

In [ ]:
submission.shape, submission.index, submission.duplicated().sum(), submission.head()

In [ ]:
submission['Survived'].value_counts(),submission.isna().sum()

In [ ]:
#submission.to_csv("asis_submission.csv")
# sumitting asis = 0.76555
#0    266
#1    152

In [ ]:
data.isna().sum()

In [ ]:
display(data[data['source']=="train"].isna().sum())
display(data.drop(columns='Survived').isna().sum())

In [ ]:
''' not working 
data[data.drop(columns='Survived').isna()]
data.isna()
'''

In [ ]:
data.info()

In [ ]:
train.shape, test.shape, data.shape, data.index

In [ ]:
data = data.reset_index().drop(columns=['index'])#,'level_0'])
data.index


In [ ]:
data

In [ ]:
data.nunique()

In [ ]:
%matplotlib inline
plt.figure(figsize=(16,30))
k=1
col_cnt = 3
row_cnt = int(np.ceil(len(data.columns)/3))
for col in data.columns:
    plt.subplot(row_cnt,col_cnt,k)
    if data[col].nunique()<25: 
        title='bar'
        data[col].value_counts().plot(kind ='bar')
    else: 
        title='hist'
        data[col].value_counts().plot(kind ='hist')
    #plt.title(title+'_'+col)
    plt.ylabel(title+'_'+col)    
    k+=1
plt.show()

In [ ]:
%matplotlib inline
j=k=0
col_cnt = 3
j=[idx for idx, col in enumerate(data.drop(columns=['PassengerId','Survived','source','Cabin']).columns) if(data[col].nunique()<25) ]
j=len(j)
row_cnt = int(np.ceil(j/3))
row_cnt+=1

fig, ax = plt.subplots(ncols=col_cnt, nrows=row_cnt, figsize=(16, row_cnt*6))
ax = ax.flatten()
tgt_var = train.columns.difference(test.columns).values[0]

for idx, col in enumerate(data.drop(columns=['PassengerId','Survived','source','Cabin']).columns) : 
    if data[col].nunique()<25 : 
        cross = pd.crosstab(data[col],data[tgt_var])      
        cross.div(cross.sum(1), axis=0).plot(kind='bar', stacked=True, ax=ax[k])
        #plt.title(col)  
    else :
        plt.subplot(row_cnt,col_cnt,k+1)
        print()
        plt.scatter(x=data[col], y=data[tgt_var].notnull())
        plt.xlabel(col) 
    k+=1        
plt.show()

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.describe(exclude=[np.float64,np.int64])

In [ ]:
data.nunique()

In [ ]:
data = data.drop(columns = ['Name','Ticket','Cabin'])
#data['Cabin'].apply(lambda x: print(x))

In [ ]:
data

In [ ]:
type(data.nunique()), type(data.value_counts()), type(data.isna().sum()), type(data.info())

In [ ]:
data.nunique(), data.value_counts(), data.isna().sum(), data.info()

In [ ]:
def combinfo(data) :
    combi = pd.DataFrame(data.nunique(),columns=['Nunique()'])
    combi2 = pd.DataFrame(data.isna().sum(),columns=['isna() count'])
    combi3 = pd.concat([combi, combi2],axis=1)

    import io
    buffer = io.StringIO()
    data.info(buf=buffer, memory_usage=False)
    s = buffer.getvalue()
    df = pd.DataFrame(s.split("\n"),columns=['info'])
    df_info = df.loc[5:len(data.columns)+5-1,'info'].str.split(n=4,expand=True).reset_index(drop=True)
    df_info.drop(columns=[0,3],axis=1,inplace=True)
    df_info.columns = ['Column','Non-Null count','Dtype']
    df_info.set_index('Column',inplace=True)

    combi5 = pd.concat([combi3, df_info],axis=1)
    return combi5

In [ ]:
import io
buffer = io.StringIO()
data.info(buf=buffer, memory_usage=False)
s = buffer.getvalue()
df = pd.DataFrame(s.split("\n"),columns=['info'])
df_info = df.loc[5:len(data.columns)+5-1,'info'].str.split(n=4,expand=True).reset_index(drop=True)
df_info.drop(columns=[0,3],axis=1,inplace=True)
df_info.columns = ['Column','Non-Null count','Dtype']
print(df_info)


In [ ]:
combinfo(data)

In [ ]:
data["Sex_num"] = data["Sex"].replace({"male":1, "female":0})
data["Embarked_num"] = data["Embarked"].replace({"S":1, "C":2, "Q":3})
#data["Fare"] = data["Fare"].fillna(data[(data['Pclass']==3)]['Fare'].median())
data["Fare"] = data["Fare"].fillna(data.groupby(['Pclass','Sex']).median()['Fare'][3,"male"])
data["Embarked_num"] = data["Embarked_num"].fillna(1)
data["Embarked"] = data["Embarked"].fillna('S')
#data["Age"] = data["Age"].fillna(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'].loc[(data['Pclass','Sex','SibSp','Parch'])])
data["Age"] = data["Age"].fillna(data["Age"].median())
#data['Age'].value_counts()


In [ ]:
#Age
#No soln
'''

conditions = "(x['discharge'].isin(x['admit'])) & (x['Num1'] >= 5) & (x['Num1'] <= 12)"
filter = df.groupby('Key').apply(lambda x: pd.eval(conditions))
'''
#data["Age"].fillna(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'].iloc[(data[['Pclass','Sex','SibSp','Parch']])])

conditions = "(x['Pclass']==1)"#" & (x['Sex']==data['Sex']) & (x['SibSp']==data['SibSp']) & (x['Parch']==data['Parch'])"
grpd = pd.DataFrame(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'])
#pd.DataFrame(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age']).apply(lambda x: print(x) if x['Pclass']==1)#pd.eval(conditions))

#grpd.apply(lambda x: print(x[1,1])) #if x.Pclass==1)
#pd.eval(conditions))
print('%'*20)
display(type(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age']))
grpd = pd.DataFrame(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'])
display(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'])
display(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'].xs((3,"male",3,1),level=(0,1,2,3)))
display(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'].loc[(3,"male",3,1)])
#df.xs(('stock1', 'velocity'), level=(1,2))
display(data.groupby(['Pclass','Sex','SibSp','Parch']).median()['Age'][3,"male",3,1])
#display(data.groupby(['Pclass','Sex','Parch']).median()['Age'])#[1,"female"])

#Analysis - inp
display(data["Age"].value_counts(), data["Age"].mean(), data["Age"].median(), data["Age"].mode() )
display(data[((data['Pclass']==3) & (data['Sex']=="male") & (data['Parch']==1))])
#.all() and (data['Sex']=="male").all()
display(data[(data['Pclass']==3)]['Fare'].describe())#, data[(data['Pclass']==3)]['Fare'].median())

display(data[data.Age.isna()])

In [ ]:
combinfo(data)

In [ ]:
#Embarked
# soln:
display(data["Embarked_num"].value_counts(), data["Embarked_num"].mode()[0])
display(data[data.Embarked_num.isna()])
display(data.groupby(['Pclass','Sex','Fare']).mean()['Embarked_num'].loc[(1,"female",80):(1,"female",90)])

#analysis -cmp
display(data["Embarked_num"].value_counts(), data["Embarked_num"].mode()[0])
display(data[data.Embarked_num.isna()])

display(pd.options.display.max_rows)
display(data.groupby(['Pclass','Sex','Embarked']).mean()['Fare'][1,"female"])
display(data.groupby(['Pclass','Sex','Embarked']).median()['Fare'][1,"female"])
#display(data.groupby(['Pclass','Sex','Fare']).loc[(1,"female")])
display(data.groupby(['Pclass','Sex','Fare']).mean()['Embarked_num'].loc[(1,"female",80)])
display(data.groupby(['Pclass','Sex','Fare']).mean()['Embarked_num'].loc[(1,"female")])
display(data.groupby(['Pclass','Sex','Fare']).mean()['Embarked_num'].loc[(1,"female",80):(1,"female",90)])

In [ ]:
# Fare - 
display(data['Fare'].describe())
display(data[data.Fare.isna()])

#soln:
display(data.groupby(['Pclass','Sex']).mean()['Fare'][3,"male"])
display(data.groupby(['Pclass','Sex']).median()['Fare'][3,"male"])

#analysis:
display(data.Fare.median())
#display(data.groupby(['Pclass','Sex']).mean()['Fare'])
#display(data.groupby(['Pclass','Sex']).median()['Fare'])
#display(data.groupby(['Pclass','Sex']).median()['Fare'].filter('Pclass==3 & Sex=="male"'))
j = data.groupby(['Pclass','Sex']).median()['Fare']
k = pd.DataFrame(data.groupby(['Pclass','Sex']).median()['Fare'])
display('j=',type(j),j.index)
display('k=',type(k),k.index, k.columns,k)
display(j[3,"male"])
#display(j.xs(('3','male')))
display(data.groupby(['Pclass','Sex']).median()['Fare'][3,"male"])
display(pd.DataFrame(data.groupby(['Pclass','Sex']).median()['Fare']).query("Pclass == 3 and Sex == 'male'"))
    #lambda x: ((x[0]==3))))
#display(data.groupby(['Pclass','Sex']).median()['Fare'].filter(lambda x: ((x.Pclass==3))))
print('@'*10)
display(data['Fare'].describe())
display(data[data.Fare.isna()])
display(data[(data['Pclass']==3)]['Fare'].describe())#, data[(data['Pclass']==3)]['Fare'].median())


In [ ]:
data['SibSp'].value_counts(), data['Parch'].value_counts(), data['Embarked'].value_counts()

In [ ]:
combinfo(data)

In [ ]:
sns.heatmap(data.drop(columns=['PassengerId','Sex','Embarked','source']).corr(),annot=True)

In [ ]:
sns.heatmap(data.corr()[['Survived']],annot=True)

In [ ]:
combinfo(data).T

In [ ]:
data.columns

In [ ]:
data_preprocessed = data.drop(columns=["Sex","Embarked_num"])
data_preprocessed.columns

In [ ]:
data_preprocessed.head()

In [ ]:
data_preprocessed.shape, data_preprocessed.index

In [ ]:
combinfo(data_preprocessed)

In [ ]:
data_preprocessed["Pclass"] = data_preprocessed["Pclass"].astype('O')
data_preprocessed["SibSp"] = data_preprocessed["SibSp"].astype('O')
data_preprocessed["Parch"] = data_preprocessed["Parch"].astype('O')
print(combinfo(data_preprocessed))
num_col = data_preprocessed.select_dtypes(include=np.number).drop(columns =['PassengerId','Survived','Sex_num']).columns
cat_col = data_preprocessed.select_dtypes(exclude=np.number).drop(columns ='source').columns
num_col, cat_col

In [ ]:
cat_col_transform = pd.get_dummies(data_preprocessed[cat_col])
cat_col_transform

In [ ]:
scaler = MinMaxScaler()
num_col_transform = pd.DataFrame(scaler.fit_transform(data_preprocessed[num_col]), columns=num_col)
num_col_transform

In [ ]:
print(data_preprocessed['PassengerId'].shape,cat_col_tranform.shape, num_col_transform.shape)
print(data_preprocessed['PassengerId'].index,cat_col_tranform.index, num_col_transform.index)
print(data_preprocessed.columns,cat_col_tranform.columns, num_col_transform.columns)

In [ ]:
data_preprocessed_cat_num = pd.concat([data_preprocessed[['PassengerId','Survived','source', 'Sex_num']],cat_col_tranform, num_col_transform], axis = 1)
data_preprocessed_cat_num

In [ ]:
combinfo(data_preprocessed_cat_num)

In [ ]:
train_preprocessed = data_preprocessed_cat_num.query("source=='train'").drop(columns='source')
test_preprocessed = data_preprocessed_cat_num.query("source=='test'").drop(columns=['source','Survived'])

train_preprocessed.shape, test_preprocessed.shape, train_preprocessed.index, test_preprocessed.index
display(train_preprocessed)
display(test_preprocessed)
train_preprocessed.to_csv("train_preprocessed_basic.csv", index = False)
test_preprocessed.to_csv("test_preprocessed_basic.csv", index = False)

In [ ]:
def print_score(y_act,y_pred,label) :
    print(label, "Accuracy score  :", accuracy_score(y_act,y_pred))
    print(label, "Recall score    :", recall_score(y_act,y_pred))
    print(label, "Precision score :", precision_score(y_act,y_pred))
    print(label, "F1 score        :", f1_score(y_act,y_pred))
    print(label, "ROC_AUC score   :", roc_auc_score(y_act,y_pred))
    print(label, "Confusion Matrix:\n", confusion_matrix(y_act,y_pred))
    print(label, "Classification Report\n", classification_report(y_act,y_pred))    
#print_score(y,y_train_pred,"Train")

In [ ]:
def build_model(model,X,y,label):
    #model = LogisticRegression()
    model.fit(X,y)
    y_train_pred = model.predict(X)
    pd.Series(data=model.coef_[0],index=X.columns).plot(kind='bar')
    print_score(y,y_train_pred,label)

In [ ]:
#with basic
X = train_preprocessed.drop(columns=["PassengerId","Survived"])
y = train_preprocessed["Survived"]
model = LogisticRegression()#(random_state=0, class_weight='balanced')
build_model(model,X,y,"Train")
test_X = test_preprocessed.drop(columns='PassengerId')
y_test_pred = model.predict(test_X).astype(int)
submission['Survived'] = y_test_pred
#submission['Survived'].replace({1:"Y",0:"N"}, inplace = True)
print(submission['Survived'].value_counts())
submission.to_csv("Loan_logistic_base v1.csv",index = False)

In [ ]:
submission.head()